In [1]:
# ATM Machine Simulation (Google Colab) — OOP + Visualization + Interactive UI
# Paste this entire block into one Colab cell and run.

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime
import random
import math

# -------------------------
# OOP core classes
# -------------------------
class Transaction:
    def __init__(self, ttype, amount, timestamp=None, meta=None):
        self.type = ttype            # 'deposit', 'withdraw', 'transfer_in', 'transfer_out', 'pin_change'
        self.amount = amount
        self.time = timestamp or datetime.now()
        self.meta = meta            # e.g., {'to_account': 'acc2'}

    def __repr__(self):
        ts = self.time.strftime("%Y-%m-%d %H:%M:%S")
        m = f" ({self.meta})" if self.meta else ""
        return f"[{ts}] {self.type.upper():12s} {self.amount:8.2f}{m}"


class BankAccount:
    def __init__(self, acc_no, initial_balance=0.0):
        self.acc_no = str(acc_no)
        self.balance = float(initial_balance)
        self.transactions = []  # list of Transaction objects
        # store history of balances for visualization
        self.balance_history = [(datetime.now(), self.balance)]

    def deposit(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit must be > 0.")
        self.balance += amount
        tx = Transaction('deposit', amount)
        self.transactions.append(tx)
        self.balance_history.append((datetime.now(), self.balance))
        return tx

    def withdraw(self, amount):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdraw must be > 0.")
        if amount > self.balance:
            raise ValueError("Insufficient funds.")
        self.balance -= amount
        tx = Transaction('withdraw', amount)
        self.transactions.append(tx)
        self.balance_history.append((datetime.now(), self.balance))
        return tx

    def add_transaction(self, tx):
        self.transactions.append(tx)
        # adjust balance if transaction reflects change
        if tx.type in ('deposit', 'transfer_in'):
            self.balance += tx.amount
        elif tx.type in ('withdraw', 'transfer_out'):
            self.balance -= tx.amount
        self.balance_history.append((datetime.now(), self.balance))

    def get_mini_statement(self, n=10):
        return list(reversed(self.transactions[-n:]))

    def get_balance_history(self):
        # returns (times, balances)
        times = [t for t,b in self.balance_history]
        balances = [b for t,b in self.balance_history]
        return times, balances


class User:
    def __init__(self, name, account: BankAccount, pin):
        self.name = name
        self.account = account
        self._pin = str(pin)

    def check_pin(self, pin):
        return str(pin) == self._pin

    def change_pin(self, old_pin, new_pin):
        if not self.check_pin(old_pin):
            raise ValueError("Old PIN incorrect.")
        if len(str(new_pin)) < 4:
            raise ValueError("PIN must be at least 4 digits.")
        self._pin = str(new_pin)
        tx = Transaction('pin_change', 0.0, meta={'by': self.name})
        self.account.transactions.append(tx)
        return True


class ATM:
    def __init__(self, bank_name="PyBank"):
        self.bank_name = bank_name
        self.users = {}   # acc_no -> User
        self.logged_in_user = None

    def add_user(self, user: User):
        self.users[user.account.acc_no] = user

    def insert_card(self, acc_no):
        acc_no = str(acc_no)
        if acc_no not in self.users:
            raise KeyError("Card/account not recognized.")
        return self.users[acc_no]

    def login(self, acc_no, pin):
        acc_no = str(acc_no)
        user = self.insert_card(acc_no)
        if user.check_pin(pin):
            self.logged_in_user = user
            return True
        else:
            return False

    def logout(self):
        self.logged_in_user = None

    def withdraw(self, amount):
        if not self.logged_in_user:
            raise PermissionError("No user logged in.")
        return self.logged_in_user.account.withdraw(amount)

    def deposit(self, amount):
        if not self.logged_in_user:
            raise PermissionError("No user logged in.")
        return self.logged_in_user.account.deposit(amount)

    def transfer(self, to_acc_no, amount):
        if not self.logged_in_user:
            raise PermissionError("No user logged in.")
        to_acc_no = str(to_acc_no)
        if to_acc_no not in self.users:
            raise KeyError("Destination account not found.")
        if to_acc_no == self.logged_in_user.account.acc_no:
            raise ValueError("Cannot transfer to same account.")
        # debit from sender
        sender_acc = self.logged_in_user.account
        recipient_user = self.users[to_acc_no]
        recipient_acc = recipient_user.account
        # perform checks
        if amount <= 0:
            raise ValueError("Amount must be > 0.")
        if amount > sender_acc.balance:
            raise ValueError("Insufficient funds.")
        # create transactions
        tx_out = Transaction('transfer_out', amount, meta={'to': to_acc_no})
        sender_acc.transactions.append(tx_out)
        sender_acc.balance -= amount
        sender_acc.balance_history.append((datetime.now(), sender_acc.balance))
        # recipient
        tx_in = Transaction('transfer_in', amount, meta={'from': sender_acc.acc_no})
        recipient_acc.transactions.append(tx_in)
        recipient_acc.balance += amount
        recipient_acc.balance_history.append((datetime.now(), recipient_acc.balance))
        return tx_out, tx_in

# -------------------------
# Setup sample bank and users
# -------------------------
atm = ATM("PyBank")

# create sample accounts with some seed balances
acc1 = BankAccount("1001", initial_balance=1500.0)
acc2 = BankAccount("1002", initial_balance=500.0)
acc3 = BankAccount("1003", initial_balance=3000.0)

user1 = User("Alice", acc1, pin="1234")
user2 = User("Bob", acc2, pin="2345")
user3 = User("Carol", acc3, pin="3456")

atm.add_user(user1)
atm.add_user(user2)
atm.add_user(user3)

# -------------------------
# UI: ipywidgets controls
# -------------------------
# Output area
main_out = widgets.Output(layout=widgets.Layout(border='1px solid black', padding='8px'))

# Card / login controls
card_select = widgets.Dropdown(options=[("Alice - 1001","1001"), ("Bob - 1002","1002"), ("Carol - 1003","1003")],
                               description="Insert Card:")
pin_box = widgets.Password(placeholder='Enter PIN', description='PIN:')
login_btn = widgets.Button(description="Login", button_style='success')
logout_btn = widgets.Button(description="Logout", button_style='warning')
login_msg = widgets.HTML("Not logged in.")

# Operation controls
balance_btn = widgets.Button(description="Check Balance")
withdraw_amt = widgets.BoundedFloatText(value=100, min=1, max=1e7, step=50, description="Amount:")
withdraw_btn = widgets.Button(description="Withdraw", button_style='danger')
deposit_amt = widgets.BoundedFloatText(value=100, min=1, max=1e7, step=50, description="Amount:")
deposit_btn = widgets.Button(description="Deposit", button_style='success')
transfer_acc = widgets.Text(value='1002', description='To Acc:')
transfer_amt = widgets.BoundedFloatText(value=50, min=1, max=1e7, step=50, description="Amount:")
transfer_btn = widgets.Button(description="Transfer", button_style='info')
mini_stmt_btn = widgets.Button(description="Mini Statement")
change_pin_old = widgets.Password(placeholder='Old PIN', description='Old PIN:')
change_pin_new = widgets.Password(placeholder='New PIN', description='New PIN:')
change_pin_btn = widgets.Button(description="Change PIN")
clear_btn = widgets.Button(description="Clear Output", button_style='')

# Visualization controls
vis_btn = widgets.Button(description="Show Visuals")
clear_vis_btn = widgets.Button(description="Clear Visuals")

# Helper to update login status display
def update_login_display():
    with main_out:
        clear_output(wait=True)
        if atm.logged_in_user:
            u = atm.logged_in_user
            print(f"Logged in: {u.name} (Acc: {u.account.acc_no})")
        else:
            print("Not logged in.")
update_login_display()

# -------------------------
# UI callbacks
# -------------------------
def on_login(b):
    acc_no = card_select.value
    pin = pin_box.value
    try:
        ok = atm.login(acc_no, pin)
        with main_out:
            clear_output(wait=True)
            if ok:
                print(f"✅ Login successful — Welcome {atm.logged_in_user.name}!")
            else:
                print("❌ Incorrect PIN.")
    except KeyError as e:
        with main_out:
            clear_output(wait=True)
            print("❌ Card not recognized.")
    update_login_display()

def on_logout(b):
    atm.logout()
    with main_out:
        clear_output(wait=True)
        print("🔒 Logged out.")
    update_login_display()

def on_check_balance(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        bal = atm.logged_in_user.account.balance
        print(f"💰 Current Balance: ₹ {bal:,.2f}")

def on_withdraw(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        amt = withdraw_amt.value
        try:
            tx = atm.withdraw(amt)
            print(f"✅ Withdrawn ₹{amt:,.2f}. New balance: ₹{atm.logged_in_user.account.balance:,.2f}")
        except Exception as e:
            print("❌", str(e))

def on_deposit(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        amt = deposit_amt.value
        try:
            tx = atm.deposit(amt)
            print(f"✅ Deposited ₹{amt:,.2f}. New balance: ₹{atm.logged_in_user.account.balance:,.2f}")
        except Exception as e:
            print("❌", str(e))

def on_transfer(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        to_acc = transfer_acc.value.strip()
        amt = transfer_amt.value
        try:
            tx_out, tx_in = atm.transfer(to_acc, amt)
            print(f"✅ Transfer successful. Sent ₹{amt:,.2f} to {to_acc}.")
            print(f"   New balance: ₹{atm.logged_in_user.account.balance:,.2f}")
        except Exception as e:
            print("❌", str(e))

def on_mini_statement(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        stm = atm.logged_in_user.account.get_mini_statement(10)
        if not stm:
            print("No transactions yet.")
            return
        print("=== Mini Statement (most recent first) ===")
        for t in stm:
            print(t)

def on_change_pin(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        old = change_pin_old.value
        new = change_pin_new.value
        try:
            atm.logged_in_user.change_pin(old, new)
            print("✅ PIN changed successfully.")
        except Exception as e:
            print("❌", str(e))

def on_clear(b):
    with main_out:
        clear_output(wait=True)
    update_login_display()

def on_show_visuals(b):
    with main_out:
        clear_output(wait=True)
        if not atm.logged_in_user:
            print("❌ Please login first.")
            return
        acc = atm.logged_in_user.account
        times, balances = acc.get_balance_history()
        # convert times to simple numeric indices for plotting
        indices = list(range(len(balances)))
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        ax[0].plot(indices, balances, marker='o')
        ax[0].set_title("Balance over time")
        ax[0].set_xlabel("Event index")
        ax[0].set_ylabel("Balance (₹)")

        # histogram of transaction types
        types = [tx.type for tx in acc.transactions]
        if types:
            uniq = {}
            for t in types:
                uniq[t] = uniq.get(t,0) + 1
            ax[1].bar(list(uniq.keys()), list(uniq.values()))
            ax[1].set_title("Transaction counts by type")
        else:
            ax[1].text(0.5, 0.5, "No transactions yet", ha='center', va='center')
            ax[1].axis('off')

        plt.tight_layout()
        plt.show()
        print(f"Displayed visuals for {atm.logged_in_user.name} (Acc: {acc.acc_no})")

def on_clear_visuals(b):
    with main_out:
        clear_output(wait=True)
    update_login_display()

# wire up buttons
login_btn.on_click(on_login)
logout_btn.on_click(on_logout)
balance_btn.on_click(on_check_balance)
withdraw_btn.on_click(on_withdraw)
deposit_btn.on_click(on_deposit)
transfer_btn.on_click(on_transfer)
mini_stmt_btn.on_click(on_mini_statement)
change_pin_btn.on_click(on_change_pin)
clear_btn.on_click(on_clear)
vis_btn.on_click(on_show_visuals)
clear_vis_btn.on_click(on_clear_visuals)

# -------------------------
# Layout
# -------------------------
left_col = widgets.VBox([
    widgets.HTML("<h3>Card & Authentication</h3>"),
    card_select,
    pin_box,
    widgets.HBox([login_btn, logout_btn, clear_btn]),
    login_msg,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>Account Ops</h3>"),
    balance_btn,
    widgets.HBox([withdraw_amt, withdraw_btn]),
    widgets.HBox([deposit_amt, deposit_btn]),
    widgets.HTML("<b>Transfer</b>"),
    widgets.HBox([transfer_acc, transfer_amt]),
    transfer_btn,
    widgets.HTML("<hr>"),
    mini_stmt_btn,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>Security</h3>"),
    change_pin_old,
    change_pin_new,
    change_pin_btn
], layout=widgets.Layout(width='48%', padding='8px'))

right_col = widgets.VBox([
    widgets.HTML("<h3>Visuals & Tools</h3>"),
    vis_btn,
    clear_vis_btn,
    widgets.HTML("<hr>"),
    widgets.HTML("<h4>Accounts Overview (sample)</h4>"),
    widgets.HTML(f"Alice - 1001 (pin 1234)"),
    widgets.HTML(f"Bob   - 1002 (pin 2345)"),
    widgets.HTML(f"Carol - 1003 (pin 3456)"),
    widgets.HTML("<hr>"),
    widgets.HTML("<i>Tip:</i> Use the sample cards above. PINs are shown for demo only.")
], layout=widgets.Layout(width='48%', padding='8px'))

ui = widgets.HBox([left_col, right_col])
display(ui)
display(main_out)

# initialize demo transactions to make visuals interesting
def seed_demo_activity():
    # random small deposits/withdrawals for each account
    for acc in [acc1, acc2, acc3]:
        for _ in range(random.randint(1,4)):
            amt = random.choice([50,100,200,500])
            if random.random() < 0.6:
                try:
                    acc.deposit(amt)
                except:
                    pass
            else:
                if amt <= acc.balance:
                    try:
                        acc.withdraw(amt)
                    except:
                        pass
seed_demo_activity()

# ensure the login display shows initial message
update_login_display()


Output(layout=Layout(border='1px solid black', padding='8px'))